In [1]:
!pip install unidecode

In [2]:
import pandas as pd
from bs4 import BeautifulSoup
from unidecode import unidecode
import re
import warnings
warnings.filterwarnings('ignore')

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')
# import os
# os.chdir('/content/drive/MyDrive/my_workspace/Sprint 3/Fake_News')

ModuleNotFoundError: No module named 'google.colab'

In [4]:
# ## Set your OpenAI API key
# api_key = open('openaiapikey.txt').read()
# SKLLMConfig.set_openai_key(api_key)

In [ ]:
df = pd.read_csv("data/Philippine Fake News Corpus.csv")
df.head()

In [6]:
df['Date'] = df['Date'].str[0:10]
df['Date'] = pd.to_datetime(df['Date'], format = '%Y-%m-%d', errors = 'coerce')
df.rename(columns={'Brand': 'Source'}, inplace=True)

print(df['Date'].min())
print(df['Date'].max())

1968-07-03 00:00:00
2018-11-16 00:00:00


In [7]:
start_date = pd.to_datetime('2018-01-01')
end_date = pd.to_datetime('2018-12-31')
df_subset = df[df['Date'].notnull()]
df_subset = df_subset[(df_subset['Date']>=start_date) & (df_subset['Date']<=end_date) ]
df_subset.reset_index(drop=True, inplace=True)


In [8]:
display(df_subset.info())
display(df_subset.nunique())
display(df_subset['Label'].value_counts())
display(df_subset['Source'].value_counts())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4640 entries, 0 to 4639
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Headline  4640 non-null   object        
 1   Content   4640 non-null   object        
 2   Authors   4640 non-null   object        
 3   Date      4640 non-null   datetime64[ns]
 4   URL       4640 non-null   object        
 5   Source    4640 non-null   object        
 6   Label     4640 non-null   object        
dtypes: datetime64[ns](1), object(6)
memory usage: 253.9+ KB


None

Headline    4640
Content     4640
Authors       66
Date         320
URL         4640
Source         6
Label          2
dtype: int64

Label
Credible        2967
Not Credible    1673
Name: count, dtype: int64

Source
Manila Bulletin             1664
Adobo Chronicles            1333
Manila Times                1303
Pinoytrending Altervista     151
Get Real Philippines         121
Duterte Daily Stories         68
Name: count, dtype: int64

In [9]:
def preprocess_text(text):
    if not isinstance(text, str):
        return text

    text = unidecode(text)

    pattern = re.compile(r'[\t\n]|<.*?>|!function.*;|\.igframe.*}')
    text = pattern.sub(' ', text)
    text = (
        text
        .replace('&#8217;', "'")
        .replace('&#8220;', '"')
        .replace('&#8221;', '"')
    )

    soup = BeautifulSoup(text, 'html.parser')
    cleaned_text = soup.get_text()
    return cleaned_text

df_subset['Headline'] = df_subset['Headline'].apply(preprocess_text)
df_subset['Content'] = df_subset['Content'].apply(preprocess_text)


In [11]:
df_subset.to_csv('data/Philippine Fake News Corpus - cleaned.csv', index = False)

In [6]:
df = pd.read_csv("data/Philippine Fake News Corpus - cleaned.csv")
df

,Headline,Content,Authors,Date,URL,Source,Label
0,Suicide bomber kills religious official in Afg...,A suicide bomber in the eastern Afghan city of...,[],2018-03-07,https://news.mb.com.ph/2018/03/07/suicide-bomb...,Manila Bulletin,Credible
1,Duterte appeals to Marawi residents to help ke...,President Duterte has vowed adequate funds to ...,[],2018-01-30,https://news.mb.com.ph/2018/01/30/duterte-appe...,Manila Bulletin,Credible
2,Hospitals blame PhilHealth anew,PRIVATE hospitals blamed the Philippine Health...,['Catherine A. Modesto'],2018-06-30,https://www.manilatimes.net/hospitals-blame-ph...,Manila Times,Credible
3,UN Palestinian refugee agency 'part of the pro...,PresseThe UN agency for Palestinian refugees i...,[],2018-05-17,https://news.mb.com.ph/2018/05/17/un-palestini...,Manila Bulletin,Credible
4,Robredo turned over livelihood subsidies to ab...,Through Vice President Leni Robredo's Angat Bu...,[],2018-03-09,https://news.mb.com.ph/2018/03/09/robredo-turn...,Manila Bulletin,Credible
...,...,...,...,...,...,...,...
4635,Plan to diversify elite NYC schools draws fire...,A plan to diversify New York City's most elite...,[],2018-06-09,https://news.mb.com.ph/2018/06/09/plan-to-dive...,Manila Bulletin,Credible
4636,Tens of thousands protest in Spain over gang r...,PresseTens of thousands of people took to the ...,[],2018-04-28,https://news.mb.com.ph/2018/04/28/tens-of-thou...,Manila Bulletin,Credible
4637,'Luis' exits but monsoon rains to continue,Tropical depression Luis blew out of the count...,['Divina Nova Joy Dela Cruz'],2018-08-25,https://www.manilatimes.net/luis-exits-but-mon...,Manila Times,Credible
4638,"28,000 families affected by floods receive aid","A TOTAL of 132,259 individuals from 28,101 fam...",['Mary Gleefer F. Jalea'],2018-08-15,https://www.manilatimes.net/28000-families-aff...,Manila Times,Credible


In [13]:
from ast import literal_eval
df['Authors'] = df['Authors'].apply(literal_eval)
df['Authors'][2][0]

'Catherine A. Modesto'

In [14]:
import itertools
authors = list(set(list(itertools.chain.from_iterable(df['Authors']))))
authors

['The Manila Times',
 'Neil Alcober',
 'Catherine Rampell',
 'Ma. Reina Leanne Tolentino',
 'Yvonne Fernandez',
 'Jove Moya',
 'Iza Iglesias',
 'Neil Jayson Servallos',
 'Jomar Canlas',
 'Glee Jalea',
 'James Konstantin Galvez',
 'Reginan Kurt Ivan Abang',
 'Javier J. Ismael',
 'Benjie Vergara',
 'Nelson Badilla',
 'Bernadette E. Tamayo',
 'Dempsey Reyes',
 'Philippines News Agency',
 'Francis Earl Cueto',
 'Kenneth Hare Hernandez',
 'Lisbet K. Esmael',
 'Jaime Pilapil',
 'Mayvelin U. Caraballo',
 'Divina Nova Joy Dela Cruz',
 'Ma. Lisbet K. Esmael',
 'Al Jacinto',
 'Ashley Jose',
 'Eireene Jairee Gomez',
 'Julmunir I. Jannaral',
 'Kimberly Malait',
 'Mt Admin',
 'Leander C. Domingo',
 'Dexter A. See',
 'Roy Narra',
 'Reicelene Joy Ignacio',
 'Athena Lucasan',
 'Aric John Sy Cua',
 'Arlo Custodio',
 'Jefferson Antiporda',
 'Ralph Villanueva',
 'Patrick Roxas',
 'View All Posts Pinoytrending',
 'Edwin U. Villanueva',
 'William Jun Garcia',
 'Mary Gleefer F. Jalea',
 'Oratio Imperata',
 

In [ ]:
brands = (df['Source'].unique()).tolist()
brands

['Manila Bulletin',
 'Manila Times',
 'Adobo Chronicles',
 'Pinoytrending Altervista',
 'Get Real Philippines',
 'Duterte Daily Stories']

In [ ]:
print(df['Date'].min())
print(df['Date'].max())

2018-01-01
2018-11-16


In [ ]:
df['Label'].value_counts()

Label
Credible        2967
Not Credible    1673
Name: count, dtype: int64

In [ ]:


import pandas as pd
import os
import itertools
from ast import literal_eval
# print(os.getcwd())
# os.chdir(r'C:\Users\bea.reyes\Documents\x')
# df = pd.read_csv('Book1.csv')
df['Authors'] = df['Authors'].apply(literal_eval)

source_author_df = df[['Source', 'Authors']]
source_author_df  = source_author_df.explode('Authors')
source_author_df = source_author_df.drop_duplicates()
sources = df['Source'].unique()

source_author_df = source_author_df.sort_values(by = ['Authors'], ascending=True, na_position='last')
source_author_df['Authors'] = source_author_df['Authors'].fillna(source_author_df['Source'] +" (No Author)")
authors = source_author_df['Authors'].unique()



# source_filter = ['A', 'B']
authors_filtered = source_author_df[source_author_df['Source'].isin(source_filter)]
authors_filtered = authors_filtered['Authors'].unique()



df_source_count = df.groupby(['Source']).size().to_frame('article_count')
df_source_count.sort_values(by = 'article_count', ascending = False)

df_author_count = df.explode('Authors')
df_author_count['Authors'] = df_author_count['Authors'].fillna(df_author_count['Source'] +" (No Author)")
df_author_count = df_author_count.groupby(['Source','Authors']).size().to_frame('article_count')
df_author_count.sort_values(by = 'article_count', ascending = False)


# import streamlit as st
# import altair as alt
# from vega_datasets import data

# # Load sample data (you can replace this with your own dataset)
# source = data.cars()

# # Create a horizontal bar chart with Altair
# chart = alt.Chart(source).mark_bar().encode(
#     x='Horsepower:Q',
#     y=alt.Y('Name:N', sort='-x'),  # Sorting bars by Horsepower descending
#     tooltip=['Name:N', 'Horsepower:Q', 'Year:N']  # Add 'Year' to tooltip
# ).properties(
#     width=600,  # Set width to make it scrollable
#     height=alt.Step(20)  # Adjust height to control bar thickness
# ).interactive()  # Enable interaction (e.g., tooltips)

# # Display the chart using Altair's native rendering in Streamlit
# st.write(chart)

In [32]:
import openai
from openai import OpenAI
from skllm.config import SKLLMConfig

api_key = open('openaiapikey.txt').read()
SKLLMConfig.set_openai_key(api_key)
client = OpenAI(api_key=api_key)

article = df
article


,Headline,Content,Authors,Date,URL,Source,Label
0,Suicide bomber kills religious official in Afg...,A suicide bomber in the eastern Afghan city of...,[],2018-03-07,https://news.mb.com.ph/2018/03/07/suicide-bomb...,Manila Bulletin,Credible
1,Duterte appeals to Marawi residents to help ke...,President Duterte has vowed adequate funds to ...,[],2018-01-30,https://news.mb.com.ph/2018/01/30/duterte-appe...,Manila Bulletin,Credible
2,Hospitals blame PhilHealth anew,PRIVATE hospitals blamed the Philippine Health...,['Catherine A. Modesto'],2018-06-30,https://www.manilatimes.net/hospitals-blame-ph...,Manila Times,Credible
3,UN Palestinian refugee agency 'part of the pro...,PresseThe UN agency for Palestinian refugees i...,[],2018-05-17,https://news.mb.com.ph/2018/05/17/un-palestini...,Manila Bulletin,Credible
4,Robredo turned over livelihood subsidies to ab...,Through Vice President Leni Robredo's Angat Bu...,[],2018-03-09,https://news.mb.com.ph/2018/03/09/robredo-turn...,Manila Bulletin,Credible
...,...,...,...,...,...,...,...
4635,Plan to diversify elite NYC schools draws fire...,A plan to diversify New York City's most elite...,[],2018-06-09,https://news.mb.com.ph/2018/06/09/plan-to-dive...,Manila Bulletin,Credible
4636,Tens of thousands protest in Spain over gang r...,PresseTens of thousands of people took to the ...,[],2018-04-28,https://news.mb.com.ph/2018/04/28/tens-of-thou...,Manila Bulletin,Credible
4637,'Luis' exits but monsoon rains to continue,Tropical depression Luis blew out of the count...,['Divina Nova Joy Dela Cruz'],2018-08-25,https://www.manilatimes.net/luis-exits-but-mon...,Manila Times,Credible
4638,"28,000 families affected by floods receive aid","A TOTAL of 132,259 individuals from 28,101 fam...",['Mary Gleefer F. Jalea'],2018-08-15,https://www.manilatimes.net/28000-families-aff...,Manila Times,Credible


In [150]:
def generate_response(content, prompt):
    response = client.chat.completions.create(
        model='gpt-3.5-turbo', 
        messages=[
            {'role': 'system', 
             'content': 
             f"Perform the specified tasks based on this article:\n\n{article}"},
            {'role': 'user', 'content': prompt}
        ]
    )
    return response.choices[0].message.content

def identify_entities(article):
    prompt = f"Identify only the person and organization entities mentioned in the article along with the article's sentiment on each mentioned entity (whether Positive, Neutral or Negative)."
    entity = generate_response(article, prompt)
    return entity


entity = identify_entities(article['Content'].values[0])
entity


'Article: Duterte appeals to Marawi residents to help keep rebellion out of Mindanao\n\nEntities and Sentiments:\n1. Person: Duterte\n   Sentiment: Positive\n\n2. Organization: Marawi residents\n   Sentiment: Positive\n\n3. Organization: Mindanao\n   Sentiment: Neutral'

In [108]:
article['Content'].values[0]

'A suicide bomber in the eastern Afghan city of Jalalabad killed a senior religious official and his bodyguard on Wednesday and wounded at least 11 people, officials said.The suicide bomber approached Abdul Zaher Haqqani, head of the Nangarhar provincial director of religious affairs and the Haj pilgrimage, and detonated his explosive vest, Ataullah Khogyani, the provincial governor\'s spokesman, said."He was coming off a bridge when the attacker approached him and blew himself up close to his car," Khogyani added.There was no immediate claim of responsibility for the attack. Nangarhar, on the border with Pakistan, is a volatile province which has become one of the main bases for Islamic State militants who fight both the government and the Taliban.'

In [198]:
def identify_entities(article):
    prompt = f"""
    Return a list of persons and organization entities mentioned in the article
        - Entities must be listed in a comma-separated list
        - Return the list only. 
        - Example: Elon Musk,Rodrigo Duterte,PhilHealth

    """
    entities = generate_response(article, prompt)
    return entities
        
        
entities

'suicide bomber, President Duterte, Marawi residents, Leni Robredo, PhilHealth, UN Palestinian refugee agency'

In [200]:
entities = identify_entities(article['Content'].values[0])
try:
    entities_ = [ent.strip() for ent in entities.split(',')]
except:
    entities_ = entities
entities_

['suicide bomber',
 'President Duterte',
 'Marawi residents',
 'Leni Robredo',
 'PhilHealth',
 'UN Palestinian refugee agency']

In [240]:
def identify_entity_sentiment(article, entities):
    prompt = f"""
    Identify the article's sentiment on each mentioned entity whether POSTITIVE, NEUTRAL or NEGATIVE:\n\n{entities}.
        
        - List them in a semi-colon-separated format along with their sentiment, POSTITIVE, NEUTRAL or NEGATIVE. 
        - Return List Only
        - Example: Elon Musk,NEGATIVE; Rodrigo Duterte,POSITIVE; PhilHealth,'NEUTRAL'
    """
    
    
    sentiment = generate_response(article, prompt)
    return sentiment
entity_sentiment = identify_entity_sentiment(article, entities)
entity_sentiment


'suicide bomber,NEGATIVE; President Duterte,POSITIVE; Marawi residents,POSITIVE; Leni Robredo,POSITIVE; PhilHealth,NEGATIVE; UN Palestinian refugee agency,NEUTRAL'

In [268]:
# try:
entity_positive = []
entity_negative = []
entity_neutral = []

for ent_sen in entity_sentiment.split(';') :
    for ent,sen in [ent_sen.split(',')]:
        if sen == 'POSITIVE':
            entity_positive = entity_positive + [ent]
        elif sen == 'NEGATIVE':
            entity_negative = entity_negative  + [ent]
        else:
            entity_neutral = entity_neutral  + [ent]

print(entity_positive)
print(entity_negative)
print(entity_neutral)

[' President Duterte', ' Marawi residents', ' Leni Robredo']
['suicide bomber', ' PhilHealth']
[' UN Palestinian refugee agency']


TypeError: list indices must be integers or slices, not str